# Getting Data from TWITTER 
CEOs / Founders:
- Tesla: https://twitter.com/elonmusk
- Amazon: https://twitter.com/ajassy 
- Apple: https://twitter.com/tim_cook
- Microsoft: https://twitter.com/satyanadella
- Google: https://twitter.com/sundarpichai




# Getting Twitter API keys

In order to access Twitter Streaming API, we need to get 4 pieces of information from Twitter: API key, API secret, Access token and Access token secret. Follow the steps below to get all 4 elements:

- Create a Twitter account if you do not already have one.
- Go to https://apps.twitter.com/ and log in with your Twitter credentials.
- Click **Create New App**
- Fill out the form, agree to the terms, and click **Create your Twitter application**
- In the next page, click on **API keys** tab, and copy your **API key** and **API secret**.
- Scroll down and click **Create my access token**, and copy your **Access token** and **Access token secret**.


# Connecting to Twitter Streaming API and Download Data using `Tweepy`

We will be using a Python library called **Tweepy** to connect to **Twitter Streaming API** and downloading the data. If you don't have Tweepy installed in your machine, go to this link ([github.com/tweepy/tweepy](https://github.com/tweepy/tweepy)), and follow the installation instructions.

To install, simply launch **Terminal** and type:
- pip install tweepy 
- **[or]** sudo pip install tweepy 

In [ ]:
# Variables that contains the user credentials to access Twitter API 
access_token = '1490688062170361857-xRop5LkXiLbTvJBQtaTRfOyHMK1eCl'     # PLEASE USE YOUR OWN
access_token_secret = '477It06qFkmEYNyAAkdmta8CTFQzbkf20SQmXt9WaDm9S'  # PLEASE USE YOUR OWN
consumer_key = 'n8JwptFiouqlI6kmefJu2mbUt'                              # PLEASE USE YOUR OWN
consumer_secret = 'VgJPQZLxjPKA8SjWGIuTSx2FD79rAek1Q7n8BDcXHTj7WT7eT1'  # PLEASE USE YOUR OWN
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAAjoYwEAAAAAq%2FlCuLSR92KunbLIz4YhiPHp5kk%3D8cxT1rofrcQr30GmMxzUFnBQzKQitRKtO8eX1ftI2yOrIxtLH1'

In [ ]:
!pip install tweepy

In [ ]:
import tweepy
from tweepy import OAuthHandler

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)
api

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
## Function to get tweets

def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #initialize a list to hold all the tweepy Tweets
    alltweets = []
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)
    
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))
    
    #transform the tweepy tweets into a 2D array that will populate the csv	
    outtweets = [[screen_name, tweet.id_str, tweet.created_at, tweet.text, tweet.retweet_count, tweet.favorite_count] for tweet in alltweets]
    tweets_df=pd.DataFrame(outtweets)
    return tweets_df

    pass

In [ ]:
ceo_list = pd.DataFrame({'Twitter Handle of CEO': ['elonmusk', 'ajassy', 'tim_cook', 'satyanadella', 'sundarpichai']})
ceo_list.head()

,Twitter Handle of CEO
0,elonmusk
1,ajassy
2,tim_cook
3,satyanadella
4,sundarpichai


In [ ]:
ceo_tweets = pd.DataFrame(columns=['username', 'id', 'date', 'tweet', 'retweets','likes'])
                                   
for handle in ceo_list["Twitter Handle of CEO"]:
    ceo_tweets_temp = get_all_tweets("@"+handle)
    ceo_tweets_temp.columns = ['username', 'id', 'date', 'tweet','retweets','likes']
    
    ceo_tweets = ceo_tweets.append(ceo_tweets_temp, ignore_index=True)

getting tweets before 1496564151493304323
...201 tweets downloaded so far
getting tweets before 1489841166652780546
...401 tweets downloaded so far
getting tweets before 1482513670777102339
...401 tweets downloaded so far
getting tweets before 1184217914964172800
...379 tweets downloaded so far
getting tweets before 586887523235803135
...379 tweets downloaded so far
getting tweets before 1308432860450230271
...400 tweets downloaded so far
getting tweets before 1186448637938798591
...600 tweets downloaded so far
getting tweets before 1087818521432784897
...800 tweets downloaded so far
getting tweets before 945302021648793599
...1000 tweets downloaded so far
getting tweets before 784423087940866047
...1200 tweets downloaded so far
getting tweets before 505355646212902912
...1287 tweets downloaded so far
getting tweets before 381131235247923200
...1287 tweets downloaded so far
getting tweets before 1349777519352766464
...400 tweets downloaded so far
getting tweets before 12204516429964165

In [ ]:
ceo_tweets.to_csv('ceo_tweets.csv', index= False)

In [ ]:
ceo_tweets

,username,id,date,tweet,retweets,likes
0,@elonmusk,1496564151493304324,2022-02-23 19:14:33,@TSLAFanMtl @DoctorJack16 I didn’t start the f...,4292,28976
1,@elonmusk,1496555372773060616,2022-02-23 18:39:40,@DoctorJack16 Building a case is exactly what ...,3503,30933
2,@elonmusk,1496554977661231107,2022-02-23 18:38:05,@ziqi37 @teslaownersSV @SECGov Exactly,446,4668
3,@elonmusk,1496525653298012167,2022-02-23 16:41:34,@Rainmaker1973 @gunsnrosesgirl3 Would be cool ...,304,7461
4,@elonmusk,1496416738304565252,2022-02-23 09:28:47,@WholeMarsBlog Yeah,288,7611
...,...,...,...,...,...,...
5176,@sundarpichai,1517273958,2009-04-14 15:15:40,Happy New Year http://tinyurl.com/d3dj3z,0,2
5177,@sundarpichai,1435641821,2009-04-02 01:50:55,"enjoying 3d browsing with Chrome, it is awesom...",0,2
5178,@sundarpichai,1205396897,2009-02-13 05:08:29,thinking abt twittering again....,0,4
5179,@sundarpichai,774770442,2008-03-21 03:26:16,enjoying a rainy day in beijing (clears out al...,1,5
